# Computing the hypervolume indicator of a Pareto frontier
This program computes the volume under a frontier of Pareto-optimal solutions to a multi-criterion optimization model, which provides a measure of the conflict among the objectives.

In [33]:
import pandas as pd
import numpy as np
import time

### Preparation for volume computation algorithm

In [45]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("../solutionSets/3d/ClimateChange_E85/climateChange_EfficientSolutions_E85.csv")

In [46]:
# Hard-coded manipulations to tidy-up sols

sols = sols.drop(["SolutionIndex","Frontier"],axis=1) #remove solution index and frontier columns

In [47]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [48]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the sols dataframe

objSenses = {}

In [49]:
# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 0
objSenses["Northern Spotted Owl Habitat (ha)"] = 1
#objSenses["PER"] = 0
#objSenses

In [50]:
# Get the objectives' bounds

objBounds = {}
for obj,sense in objSenses.items():
    objBounds[obj] = [sols[obj].min(),sols[obj].max()]

In [51]:
# Get each objective's ideal value

idealObjVals = {}
for obj,bounds in objBounds.items():
    idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max

In [52]:
# Get each objective's worst value

worstObjVals = {}
for obj,bounds in objBounds.items():
    worstObjVals[obj] = bounds[not(objSenses[obj])]

In [53]:
# Compute the non-normalized volume of the objective space

objSpaceVolume_nonNorm = 1
for obj,bounds in objBounds.items():
    objSpaceVolume_nonNorm *= bounds[1] - bounds[0]

In [54]:
# Normalize the objective space by converting each value to the relative achievement along its axis:

#   distance from the worst case value
# ---------------------------------------
# total distance spanned by the objective

# Objective space is now in [0,1]^N where N is number of objectives


for obj,bounds in objBounds.items():
    sols[obj] = sols[obj].apply(
        lambda x: \
        abs(x - worstObjVals[obj]) / \
        (bounds[1]-bounds[0]))

### Algorithm to compute volume of frontier

#### "Add-the-sides" frontier volume computation method

For a mathematical description of the below algorithm, see ./algorithmWriteUp/frontierVolumeComputation_writeup.pdf

In [56]:
# Initialize frontier volume

frontierVolume = 0

In [26]:
# Sort the dataframe in descending order from first column
# (always descending, since objectives normalized bad->good = 0->1)

sols = sols.sort_values(by=[objs[0]],ascending=False)

In [57]:
# Create an empty dataframe to hold the solutions whose volumes have already been accounted for,
# adding a column to hold its subdimensional volume contribution,
# and a column to indicate whether it is a boundary solution in sub-dimensional space


completedFrontierPoints = sols[sols[objs[0]]<0]
completedFrontierPoints["SubDimContribution"] = pd.Series()
completedFrontierPoints["BoundarySolution"] = pd.Series()

In [28]:
# Get list of all non-primary dimensions

dims_secondary = [col for col in sols.columns if col not in [objs[0]]]

In [29]:
def getSideVolInSubDim(dim,frontierPoint,completedFrontierPoints):
    
    sideVolInSubDim = 0
    
    # Get the sorted list of boundary solutions with dim component larger than the current point
    sideSols_dim = completedFrontierPoints.loc[(completedFrontierPoints["BoundarySolution"]>0) & \
                                              (completedFrontierPoints[dim] > frontierPoint[dim])]\
                                            .sort_values(by=[dim],ascending=True)
    # If there are none, there are no sides to add
    if sideSols_dim.empty:
        return sideVolInSubDim
    
    otherSecondaryDims = [d for d in dims_secondary if d != dim]
    prevDimComponent = frontierPoint[dim]
    currDimComponent = 0
    for idx,row in sideSols_dim.iterrows():
        currDimComponent = row[dim]
        dimDelta = currDimComponent - prevDimComponent
        sideVolInSubDim += dimDelta * row[otherSecondaryDims].product()
        prevDimComponent = currDimComponent
    
    return sideVolInSubDim
        

In [30]:
def getSubDimVolumeFromFrontierPoint(frontierPoint,completedFrontierPoints):
    # Get a solution's sub-D volume back to the origin
    subDimContrib = frontierPoint[dims_secondary].product()
    # Subtract everything pre-existing away from it
    subDimContrib -= completedFrontierPoints["SubDimContribution"].sum()
    # Add back in the sides
    for dim in dims_secondary:
        subDimContrib += getSideVolInSubDim(dim,frontierPoint,completedFrontierPoints)
    
    return subDimContrib

In [31]:
def getFrontierVolume(initFrontierVolume, remainingFrontierPoints, completedFrontierPoints):
    if len(remainingFrontierPoints) == 0:
        return initFrontierVolume
    else:
        # next solution to add to frontier vol
        currSol = remainingFrontierPoints.iloc[0]
        # will always be nondominated in sub-D space
        currSol["BoundarySolution"] = True
        # change boundary status of points that the current solution dominates:
        completedFrontierPoints.loc[np.all([completedFrontierPoints[obj] < currSol[obj] for obj in dims_secondary]\
                                           ,axis=0),\
                                    "BoundarySolution"] = False
        # Get the sub-D volume for the current solution
        currSol["SubDimContribution"] = getSubDimVolumeFromFrontierPoint(currSol,completedFrontierPoints)
        # Update the frontier volume
        initFrontierVolume += currSol["SubDimContribution"] * currSol[objs[0]]
        # update the points added to the frontier
        completedFrontierPoints = completedFrontierPoints.append(currSol,ignore_index=True)
        # update the remaining points
        remainingFrontierPoints = remainingFrontierPoints.drop(currSol.name)
        # recursive call
        return getFrontierVolume(initFrontierVolume,remainingFrontierPoints,completedFrontierPoints)

In [32]:
start = time.clock()
frontierVolume = getFrontierVolume(frontierVolume,sols,completedFrontierPoints)
end = time.clock()
print ("Elapsed time: " + (str(end-start)))
print ("Frontier Volume: " + str(frontierVolume))

Elapsed time: 1.3993453481141767
Frontier Volume: 0.746556764487
